## Ejemplo análisis de una sección

In [8]:
from mento import Concrete_ACI_318_19, SteelBar, Forces, RectangularConcreteBeam
from mento import mm, cm, kN,MPa
import pandas as pd

concrete= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
steelBar= SteelBar(name="ADN 420", f_y=420*MPa) 
section = RectangularConcreteBeam(label="101",concrete=concrete,steel_bar=steelBar,width=20*cm, height=40*cm)
f = Forces(V_z=86*kN)
section.set_transverse_rebar(n_stirrups=1, d_b=8*mm, s_l=25*cm) 
section.check_shear()
# section.results
# section.design_shear(f)

,"Av,min","Av,req",Av,Vu,ØVc,ØVs,ØVn,ØVmax,Vu<ØVmax,Vu<ØVn,FUv
0,0.00 cm²/m,0.00 cm²/m,4.02 cm²/m,0.00 kN,44.71 kN,44.79 kN,89.50 kN,223.57 kN,True,True,0.00


In [14]:
from mento import Concrete_ACI_318_19, SteelBar, Forces, RectangularConcreteBeam
from mento import mm, cm, kN,MPa, m, inch, ft, kNm
from pandas import DataFrame
from pint.facets.plain import PlainQuantity
from typing import List

# Set future-proof option for pandas behavior
pd.set_option('future.no_silent_downcasting', True)

class BeamSummary:
    def __init__(self: DataFrame, concrete, steel_bar, beam_list: DataFrame):
        self.concrete = concrete
        self.steel_bar = steel_bar
        self.beam_list = beam_list
        self.check_and_process_input()
        self.convert_to_beams()

    def check_and_process_input(self) -> None:
        # Separate the header, units, and data
        self.units_row = self.beam_list.iloc[0].tolist()  # Second row (units)
        data = self.beam_list.iloc[1:].copy()  # Data rows (after removing the units row)

        # Convert NaN in units to "dimensionless"
        self.units_row = ['' if pd.isna(unit) else unit for unit in self.units_row]

        # Validate the units row
        self.validate_units(self.units_row)

        # Convert NaN to 0 in the data rows
        data.fillna(0, inplace=True)

        # Apply units to the corresponding columns, skipping the first column
        for i in range(1, len(self.units_row)):  # Start from the second column (index 1)
            unit_str = self.units_row[i]
            if unit_str != '':
                unit = self.get_unit_variable(unit_str)
                if isinstance(data.iloc[:, i], pd.Series):
                    data.iloc[:, i]= data.iloc[:, i].apply(lambda x: x * unit)


        # Store the processed data
        self.data = data
        print("Processed Data: Ok")

    def validate_units(self, units_row) -> None:
        valid_units = {"m", "mm", "cm", "inch", "ft", "kN", "kNm", ''}
        for unit_str in units_row:
            if unit_str and unit_str not in valid_units:
                raise ValueError(f"Invalid unit '{unit_str}' detected. Allowed units: {valid_units}")
        print("Processed Units: Ok")
    def get_unit_variable(self, unit_str) -> None:
        # Map strings to actual unit variables (predefined in the script)
        unit_map = {
            'mm': mm,
            'cm': cm,
            'm': m,
            'in': inch,
            'ft': ft,
            'kN': kN,
            'kNm': kNm,
            'MPa': MPa,
        }
        if unit_str in unit_map:
            return unit_map[unit_str]
        else:
            raise ValueError(f"Unit '{unit_str}' is not recognized.")
        
    def convert_to_beams(self) -> List:
        self.beams = []

        for index, row in self.data.iterrows():
            # Extract forces for each row
            M_y = row['My']  # Example: My in kNm
            N_x = row['Nx']  # Example: Nx in kN
            V_z = row['Vz']  # Example: Vz in kN

            # Ensure these are pint.Quantity objects with correct units
            forces = Forces(M_y=M_y, N_x=N_x, V_z=V_z)

            # Extract geometric properties of the beam (width and height)
            width = row['b']  # Example: width in cm
            height = row['h']  # Example: height in cm

            # Create a rectangular concrete beam using the extracted values
            section = RectangularConcreteBeam(label=row['Label'],
                                              concrete=self.concrete,
                                              steel_bar=self.steel_bar,
                                              width=width,
                                              height=height)        

            # Set transverse rebar (stirrups) for the beam
            n_stirrups = row['ns']  # Number of stirrups
            d_b = row['dbs']  # Diameter of rebar (mm)
            s_l = row['sl']  # Spacing of stirrups (cm)

            section.set_transverse_rebar(n_stirrups=n_stirrups, d_b=d_b, s_l=s_l)

            # Store the section and its corresponding forces
            self.beams.append({'section': section, 'forces': forces})

    def capacity(self) -> DataFrame:
        for item in beam_summary.beams:
            beam = item['section']
            forces = item['forces']
            beam.check_shear(forces)
            # Design results
            results = {
                'Av,min': A_v_min.to('cm ** 2 / m'),  # Minimum shear reinforcement area
                'Av,req': A_v_req.to('cm ** 2 / m'), # Required shear reinforcing area
                'Av': A_v.to('cm ** 2 / m'),  # Provided stirrup reinforcement per unit length
                'Vu': self._V_u.to('kN'), # Max Vu for the design
                'ØVc': phi_V_c.to('kN'),  # Concrete contribution to shear capacity
                'ØVs': phi_V_s.to('kN'),  # Reinforcement contribution to shear capacity
                'ØVn': self._phi_V_n.to('kN'),  # Total shear capacity
                'ØVmax': phi_V_max.to('kN'),  # Maximum shear capacity
                'Vu<ØVmax': max_shear_ok,  # Check if applied shear is within max shear capacity
                'Vu<ØVn': self._V_u <= self._phi_V_n,  # Check if applied shear is within total capacity
                "FUv" :  self._FUv
            }
            return pd.DataFrame([results], index=[0])

conc= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
steel= SteelBar(name="ADN 420", f_y=420*MPa) 
# input_df = pd.read_excel('Mento-Input.xlsx', sheet_name='Beams', usecols='B:R', nrows=9, skiprows=4)
data = {'Label': ['', 'V101', 'V102', 'V103', 'V104'],
        'b': ['cm', 20, 20, 20, 20],
        'h': ['cm', 30, 30, 40, 40],
        'Nx': ['kN', 10, 15, 20, 25],
        'Vz': ['kN', 20, 25, 30, 35],
        'My': ['kNm', 30, 35, 40, 45],
        'ns': ['', 1.0, 1.0, 1.0, 1.0],
        'dbs': ['mm', 6, 8, 6, 8],
        'sl': ['cm', 20, 20, 20, 20],
        'n1': ['', 2.0, 2.0, 2.0, 2.0],
        'db1': ['mm', 12, 12, 12, 12],
        'n2': ['', 1.0, 0.0, 1.0, 0.0],
        'db2': ['mm', 10, 0, 10, 0],
        'n3': ['', 2.0, 0.0, 2.0, 0.0],
        'db3': ['mm', 12, 0, 0, 0],
        'n4': ['', 1.0, 0.0, 1.0, 0.0],
        'db4': ['mm', 10, 0, 0, 0]}

input_df = pd.DataFrame(data)
input_df
beam_summary = BeamSummary(concrete=conc, steel_bar=steel, beam_list=input_df)
# beam_summary.data
beam_summary.capacity()
# beam_summary.check()
# beam_summary.design()


Processed Units: Ok
Processed Data: Ok


In [102]:
v = [1*cm ,2*cm ,3*kN]
for i in range(2, len(v)): print(i)
df = pd.DataFrame(v)
df
a = PlainQuantity(1)
a

2


<Quantity(1, 'dimensionless')>

In [45]:
from mento.results import SectionSummary
from typing import Optional, Dict, Any, cast
import pint
from pint.facets.plain import PlainQuantity

class RecBeamData:
    def __init__(self, name, b, h, as_inf, as_sup, A_v, Vu, Mu):
        self.name = name  # Beam name, e.g., 'VF-01'
        self.b = b  # Width of the beam in cm
        self.h = h  # Height of the beam in cm
        self.as_inf = as_inf  # Lower reinforcement (string)
        self.as_sup = as_sup  # Upper reinforcement (string)
        self.A_v = A_v  # Shear reinforcement (string)
        self.Vu = Vu # Ultimate shear
        self.Mu = Mu # Ultimate bending moment

conc= Concrete_ACI_318_19(name="C25",f_c=25*MPa) 
stel= SteelBar(name="ADN 420", f_y=420*MPa)
beam_list = [
    RecBeamData("V-101", b=20*cm, h=40*cm, A_v="1Ø6mm/20cm"),
    RecBeamData("V-102", b=20*cm, h=50*cm, A_v="1Ø6mm/20cm"),
    RecBeamData("V-103", b=20*cm, h=60*cm, A_v="1Ø6mm/20cm"),
]
summary = SectionSummary(beam_list)

TypeError: RecBeamData.__init__() missing 4 required positional arguments: 'as_inf', 'as_sup', 'Vu', and 'Mu'